In [20]:
from qa_model import QASystem, Encoder, Decoder
from train import create_hparams, initialize_vocab
from train import FLAGS


In [21]:
hparams = create_hparams(FLAGS)

In [23]:
hparams.batch_size = 1
hparams.input_keep_prob = 1.0


In [15]:
val_p_file = './data/squad/val.ids.context'
val_q_file = './data/squad/val.ids.question'
val_span = './data/squad/val.span'


In [24]:
vocab, rev_vocab = initialize_vocab('./data/squad/vocab.dat')
hparams.add_hparam('vocab_size', len(vocab))

In [25]:
encoder = Encoder(size=FLAGS.state_size, vocab_dim=FLAGS.embedding_size)
decoder = Decoder(state_size=FLAGS.state_size)
qa = QASystem(encoder, decoder, hparams)

In [26]:
iterator = qa.make_dataset_iterator(val_p_file, val_q_file, val_span, batch_size=1, epoch_size=1)

In [27]:
import numpy as np
np_embeddings = np.float32(np.load('./data/squad/glove.trimmed.100.npz')['glove'])

In [28]:
context_tuple, question_tuple, span = iterator.get_next()
p, p_mask = context_tuple
q, q_mask = question_tuple
span = span


In [30]:
qa.setup_system(p, q, p_mask, q_mask, span)

(?, ?, 100) (?, ?, 100) Flattening
(?, ?, 100) (?, ?, 100) Reconstruct
(?, ?, 100) (?, ?, 100) Flattening
(?, ?, 100) (?, ?, 100) Reconstruct
(1, ?, 1400) (1, ?, 1400) Flattening
(1, ?, 100) (1, ?, 100) Reconstruct


In [35]:
import tensorflow as tf

saver = tf.train.Saver()

sess = tf.InteractiveSession()
saver.restore(sess, "/tmp/cs224n-squad-train/model999.ckpt")

sess.run(qa.embeddings_init, feed_dict={qa.np_embeddings: np_embeddings})


INFO:tensorflow:Restoring parameters from /tmp/cs224n-squad-train/model999.ckpt


INFO:tensorflow:Restoring parameters from /tmp/cs224n-squad-train/model999.ckpt


array([[ 1.51184046,  1.83328331,  1.12435555, ...,  0.36550143,
        -0.75359511, -0.68179172],
       [ 1.42015517, -0.6002773 , -0.66393876, ...,  1.2193594 ,
        -1.34410429, -0.96097118],
       [ 0.21964514, -0.98801208, -0.15381271, ...,  2.21488619,
        -0.56242007,  0.22986133],
       ..., 
       [ 0.35914558, -0.825854  , -0.13772602, ...,  0.12259456,
         0.46273223, -0.86086357],
       [ 0.89044452,  0.4215835 , -0.29643965, ..., -1.37525487,
         0.42506322,  0.04041158],
       [-0.13890173, -0.88782078,  0.92992932, ...,  0.06739603,
         1.74034679, -0.39021042]], dtype=float32)

In [58]:
feed_dict = {
    qa.is_train: False
 }

yp1, yp2, p, q = sess.run([qa.yp1, qa.yp2, qa.p, qa.q], feed_dict=feed_dict)
    
print yp1
print yp2
print p
print q
print np.argmax(yp1), np.argmax(yp2)
print np.shape(p)
print np.max(yp1), np.max(yp2)
max_elems = np.matmul(np.transpose(yp1), yp2)

print np.unravel_index(np.argmax(max_elems), np.shape(max_elems))

[[ 0.04146253  0.02328203  0.01810483  0.01917072  0.01958251  0.01949828
   0.0191248   0.01867296  0.01836959  0.01792364  0.017403    0.0168167
   0.01639643  0.01621682  0.01602316  0.01551192  0.01525443  0.01495187
   0.01502855  0.0145721   0.01438708  0.01415383  0.01400034  0.01373407
   0.01351646  0.01347142  0.0132772   0.01302389  0.01291548  0.01301399
   0.01284705  0.01278117  0.0123946   0.01225075  0.0122525   0.01218597
   0.01203254  0.01173991  0.01159999  0.01158052  0.01134979  0.01129117
   0.01130529  0.01114008  0.01101293  0.01087849  0.01084131  0.01049471
   0.010285    0.01011071  0.01012389  0.01019453  0.01008709  0.01009129
   0.00997854  0.00988227  0.00983194  0.00964774  0.00957082  0.0094921
   0.00936988  0.00926387  0.00924258  0.00923668  0.00920125  0.00910131
   0.00900699  0.00878848  0.00894843  0.00886413  0.00886639  0.0086796
   0.00854938  0.00828797  0.00827471  0.00911004  0.01114771  0.00691099
   0.00101428]]
[[ 0.0151583   0.01890317